### Add scripts path to the notebook

In [1]:
import sys
import os

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 2\TellCo-Profitability-Opportunity-Analysis\notebooks


### Import Statements

In [3]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from scripts.database_client import DB_Client

### Initialize DB_Client

In [4]:
# obtain values form environment variables
host = os.getenv("DB_HOST")
user_name = os.getenv("DB_USER")
passowrd = os.getenv("DB_PASSWORD")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

In [5]:
db_client = DB_Client(
    host=host,
    user_name=user_name,
    password=passowrd,
    port=port,
    database_name=database
)

### Load the data from the database

In [6]:
data = db_client.dump_data()

d:\KifiyaAIM-Course\Week - 2\TellCo-Profitability-Opportunity-Analysis\scripts\database_client.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  response = pd.read_sql_query(sql=query, con=self.connection)


### Data Cleaning

In [7]:
data.isna().mean()

Bearer Id                                   0.006607
Start                                       0.000007
Start ms                                    0.000007
End                                         0.000007
End ms                                      0.000007
Dur. (ms)                                   0.000007
IMSI                                        0.003800
MSISDN/Number                               0.007107
IMEI                                        0.003813
Last Location Name                          0.007687
Avg RTT DL (ms)                             0.185525
Avg RTT UL (ms)                             0.185412
Avg Bearer TP DL (kbps)                     0.000007
Avg Bearer TP UL (kbps)                     0.000007
TCP DL Retrans. Vol (Bytes)                 0.587636
TCP UL Retrans. Vol (Bytes)                 0.644322
DL TP < 50 Kbps (%)                         0.005027
50 Kbps < DL TP < 250 Kbps (%)              0.005027
250 Kbps < DL TP < 1 Mbps (%)               0.

As described in the _user_overview_analysis.ipynb_ since the data has a lot of missing vlaues in some columns droping the rows isn't the best option. So only cleanning the features that are important/relevant to the current task is the better option. The features of interest(columns) with null values are put in a list and the cleaning only happens in those columns, just for the specific task

In [8]:
columns_of_interest = ["MSISDN/Number", "IMSI", "Bearer Id", "Start", "Start ms", "End", "End ms", "Dur. (ms)", "Dur. (ms).1"]
data[columns_of_interest].isna().mean()

MSISDN/Number    0.007107
IMSI             0.003800
Bearer Id        0.006607
Start            0.000007
Start ms         0.000007
End              0.000007
End ms           0.000007
Dur. (ms)        0.000007
Dur. (ms).1      0.000007
dtype: float64

I have decided to drop the missing values found in the given columns because the amount of data we loose is very little

In [9]:
data.dropna(subset=columns_of_interest, inplace=True)

### Tracking user engagement 

1) Group data based on the MSISDN (phone number)

In [12]:
customer_grouping = data.groupby(by="MSISDN/Number")

2) Aggregate the required data. Since every entry in the database is a session we have to group them by the phone numebr, which we did above, and then calculate the total frequency by adding the unique _Bearer Id / sessionId_, summing up the download and upload byets for the sessions, and also adding the durations per sessions

In [37]:
customer_data = customer_grouping.agg({
    "Bearer Id": "count", # count the amount of session per sim card
    "Dur. (ms)": "sum", # calculate the total duration per sim card in the data
    "Total UL (Bytes)":  "sum", #calculate the total upload per sim card
    "Total DL (Bytes)": "sum", # calculate the total download per sim card 
})

# finding the total trafic(sum between total download and uplaod)
customer_data["traffic"] = customer_data["Total UL (Bytes)"] + customer_data["Total DL (Bytes)"]

# renaming the fields
customer_data.rename(columns={
    "Bearer Id": "session_freq",
    "Dur. (ms)": "duration",
    "Total UL (Bytes)": "upload_tot",
    "Total DL (Bytes)": "download_tot"
}, inplace=True)

In [38]:
customer_data.columns

Index(['session_freq', 'duration', 'upload_tot', 'download_tot', 'traffic'], dtype='object')

3) Reporting the top 10 customers per metric

In [42]:
# find the top 10 users for session frequency
frequency_order = customer_data.sort_values(by="session_freq", ascending=False)
frequency_order.head(10)

,session_freq,duration,upload_tot,download_tot,traffic
MSISDN/Number,,,,,
3.362632e+10,18,8791927.0,669650721.0,7.301517e+09,7.971167e+09
3.361489e+10,17,9966898.0,689483001.0,8.156743e+09,8.846226e+09
3.362578e+10,17,18553754.0,729577380.0,7.770043e+09,8.499621e+09
3.365973e+10,16,4035428.0,624260321.0,7.081602e+09,7.705863e+09
3.376054e+10,15,9279434.0,703478581.0,7.811295e+09,8.514774e+09
3.367588e+10,15,4865947.0,581568792.0,7.309542e+09,7.891111e+09
3.366716e+10,13,8744914.0,566326364.0,5.052068e+09,5.618394e+09
3.376041e+10,12,5321667.0,521518890.0,6.610852e+09,7.132371e+09
3.378632e+10,12,4059652.0,466330265.0,5.155901e+09,5.622232e+09


In [44]:
# find the top 10 users for session duration
duration_order = customer_data.sort_values(by="duration", ascending=False)
duration_order.head(10)

,session_freq,duration,upload_tot,download_tot,traffic
MSISDN/Number,,,,,
3.362578e+10,17,18553754.0,729577380.0,7.770043e+09,8.499621e+09
3.361489e+10,17,9966898.0,689483001.0,8.156743e+09,8.846226e+09
3.376054e+10,15,9279434.0,703478581.0,7.811295e+09,8.514774e+09
3.362632e+10,18,8791927.0,669650721.0,7.301517e+09,7.971167e+09
3.366716e+10,13,8744914.0,566326364.0,5.052068e+09,5.618394e+09
3.366284e+10,9,6614270.0,406424372.0,3.593674e+09,4.000098e+09
3.366469e+10,9,6288730.0,402180804.0,5.279827e+09,5.682007e+09
3.360313e+10,12,6287761.0,435587631.0,4.540607e+09,4.976195e+09
3.366746e+10,6,5649882.0,266222647.0,2.334299e+09,2.600522e+09


In [45]:
# find the top 10 users by total traffic 
traffic_order = customer_data.sort_values(by="traffic", ascending=False)
traffic_order.head(10)

,session_freq,duration,upload_tot,download_tot,traffic
MSISDN/Number,,,,,
3.361489e+10,17,9966898.0,689483001.0,8.156743e+09,8.846226e+09
3.376054e+10,15,9279434.0,703478581.0,7.811295e+09,8.514774e+09
3.362578e+10,17,18553754.0,729577380.0,7.770043e+09,8.499621e+09
3.362632e+10,18,8791927.0,669650721.0,7.301517e+09,7.971167e+09
3.367588e+10,15,4865947.0,581568792.0,7.309542e+09,7.891111e+09
3.365973e+10,16,4035428.0,624260321.0,7.081602e+09,7.705863e+09
3.366646e+10,11,4536757.0,405060976.0,6.903440e+09,7.308501e+09
3.376041e+10,12,5321667.0,521518890.0,6.610852e+09,7.132371e+09
3.366471e+10,11,2927785.0,471244453.0,6.400774e+09,6.872018e+09
